# Import required libraries

In [ ]:
import os  # <--- Get environmental variables
import re  # <--- Regex for data cleaning
import json  # <--- Loading JSON into Python objects
import dirtyjson  # <--- backup method for loading dirty JSON into Python objects
import asyncio  # <--- Execute tasks asynchronously
import pandas as pd  # <--- DataFrame usage for dataset operations
import numpy as np  # <--- Numpy arrays for clustering algorithms
import plotly.express as px  # <--- Fancy graphing!
from getpass import getpass  # <--- Get OpenAI API key if not stored in environmental variable
from helpers import DocumentTransformer, EvtxHandler, Filter  # <--- Custom funcs/classes for parsing/transforming EVTX data
from openai import AsyncOpenAI  # <--- Async OpenAI client
from sklearn.cluster import DBSCAN  # <--- Our clustering algorithm
from sklearn.decomposition import PCA  # <--- Princaple Component Analysis for graphing
from typing import List, Tuple  # <--- Typing to help convey variable types

# Get user input
User will need to specify OpenAI key and a source of event logs.

In [ ]:
# Get OpenAI API key from env var or else prompt for it.
openai_key = os.environ.get("OPENAI_KEY", None) or getpass("Enter your OpenAI key:")

# We are going to pull commands from EVTX files. Provide a path to EVTX files that contain Sysmon logs.
evtx_source = input("Enter a source for a EVTX file or folder that contains EVTX files: ")
print(f"EVTX Source: {evtx_source}")

# Create a EvtxHandler that can transform and filter events
These are helper classes in the helpers.py module that sits in this folder.

In [ ]:
# We want to transform EVTX records into rows that only have a couple columns.
document_transformer = DocumentTransformer.from_fields([
        ("Timestamp", "Event.System.TimeCreated.\"#attributes\".SystemTime"),
        ("Computer", "Event.System.Computer"),
        ("Provider", "Event.System.Provider.\"#attributes\".Name"),
        ("EventID", "Event.System.EventID.\"#text\"||Event.System.EventID"),
        ("CommandLine", "Event.EventData.CommandLine"),
    ])

# We only want to return EVTX records that have Event.EventData.CommandLine populated.
evtx_filter = Filter.from_pattern("Event.EventData.CommandLine")

# Create an EvtxHandler to make EVTX operations easy.
evtx_handler = EvtxHandler.from_source(evtx_source)\
    .with_transformer(document_transformer)\
    .with_filter(evtx_filter)

# Parse EventLogs into a DataFrame

In [ ]:
# Get a DataFrame that represents our EVTX data.
dataframe = evtx_handler.parse_into_dataframe()
# Show first five records
dataframe.iloc[:5]

In [ ]:
print("Total Events: {}".format(dataframe.shape[0]))
print("Unique commands found: {}".format(
    dataframe["CommandLine"].unique().shape[0]
))

# Request Embeddings from OpenAI

In [ ]:
# Create async OpenAI client
client = AsyncOpenAI(api_key=openai_key)

In [ ]:
# Create an async function to fetch embeddings for given text
async def get_embedding(
    text: str, semaphore: asyncio.Semaphore, 
    model="text-embedding-3-small", dimensions=None
) -> Tuple[str, List[float]]:
    # Use a Semaphore to keep a max number to throttle requests
    async with semaphore as sep:
        # Request dimensions if provided, otherwise send without dimensions param
        if dimensions:
            response = await client.embeddings.create(input=text, model=model, dimensions=dimensions)
        else:
            response = await client.embeddings.create(input=text, model=model)
        # Extract the embedding vector
        embedding = response.data[0].embedding
        # Return the command and it's embedding vector
        return text, embedding

# Iterate Commands Requesting the Embeddings for Each One
Remember we only want to iterate on **unique** CommandLines. 

In [ ]:
# Create a Semaphore to limit how many requests can happen at a time
semaphore = asyncio.Semaphore(25)
# Create async tasks to generate embeddings for CommandLines
tasks = [get_embedding(cmd, semaphore) for cmd in dataframe["CommandLine"].unique()]
# Get the results of our embedding generation
embedding_results = await asyncio.gather(*tasks)

In [ ]:
# Create a DataFrame from the embeddings and commands
df_embeddings = pd.DataFrame(embedding_results, columns=["cmd", "embedding_vector"])
# Show five records
df_embeddings.iloc[5:10]

In [ ]:
print(df_embeddings.iloc[0][1])

# Cluster the Embeddings

In [ ]:
# To cluster data we need to break the embeddings into an array
unique_command_lines_vectors = np.array(list(df_embeddings["embedding_vector"]))
# Collect the list of commands
command_list = df_embeddings["cmd"]

## Use the DBSCAN clustering algorithm 

In [ ]:
# Cluster the command line vectors
dbscan = DBSCAN(n_jobs=-1, min_samples=2, eps=0.56)
dbscan.fit(unique_command_lines_vectors)
print("Number of Clusters: {}".format(len(set(dbscan.labels_))))

In [ ]:
# Create a DataFrame that contains our commands and their associated clusters
clusted_commands_df = pd.DataFrame({"Command": command_list, "Cluster": dbscan.labels_})
# View cluster counts out of curiosity
clusted_commands_df.groupby(["Cluster"]).agg("count").reset_index()\
    .rename(columns={"Command": "Unique Commands in Cluster"}).transpose()

# Example Commands in a Given Cluster

In [ ]:
# What is cluster 79?
clusted_commands_df[clusted_commands_df["Cluster"]==79]

# Convert multi-dimensional vectors into 3 dimensions for graphing

In [ ]:
# Apply a Princaple Component Analysis to our multi dimensional vectors to simplify down to 3d vectors for plotting
pca = PCA(3, n_oversamples=1)
pca.fit(unique_command_lines_vectors)
three_dimensions = pca.transform(unique_command_lines_vectors)
print("Example: {} => {}".format(unique_command_lines_vectors[0], three_dimensions[0]))

# Graphing the Cluster into 3D

In [ ]:
# Function that will create a scatter plot figure of vectors in 3 dimensions
def scatter3d(data: List[Tuple[float, float, float]], labels: List[int], exclude_unclustered=False):
    """Data is a list of 3d vectors. Labels are the clusters that correlate to the a given vector.
    """
    # Create a DataFrame that has the X, Y, Z coordinates of each item.
    _df = pd.DataFrame({
        "cluster": labels,
        "x": data[:, 0],
        "y": data[:, 1],
        "z": data[:, 2]
    })
    # Exclude unclusted data if requested
    if exclude_unclustered:
        _df = _df[_df["cluster"] != -1]

    # Create a figure with our DataFrame
    fig = px.scatter_3d(
        _df,
        x='x', y='y', z='z',
        color='cluster'
    )
    fig.write_html("plot.html")
    # Return the figure
    return fig

# Plot the data
scatter3d(three_dimensions, dbscan.labels_, exclude_unclustered=True)

In [ ]:
# View an example of a cluster
clusted_commands_df[clusted_commands_df["Cluster"]==30].iloc[:20]

# Using the LLM to Assess Risk of Each Cluster
### Define Prompts
The better the prompt, the better your results will be!

In [ ]:
# The number of samples from each cluster to analyze
command_sample_size = 10
# System prompt for each OpenAI chat request
system_prompt = r"""You are a digital forensics and incident response analyst reviewing commands executed on a Windows system. 
You are also proficent with data science and understand machine learning strategies. 
You are using the DBSCAN clusterning algorithm to group commands executed. For each cluster, assess the risk of the commands used.
Commands are a sample of the given cluster. The command will be between the following tags:  <command> and </command>"""
# Chat prompt for each cluster
user_prompt_template = "Analyze the following commands given this sample of cluster {cluster_number}.\n\nYour output must be " \
"valid JSON that adheres to the JSON standard with the following format.\n\nrisk_score must be a value between 0-10." \
"\n\n<output format>\n{{\n\t\"risk_score\": <int>\n\t\"cluster_description\": <str>\n}}\n</output format>\n\n" \
"<commands to analyze>\n{command}\n</commands to analyze>"

# Using the LLM to Assess Risk of Each Cluster
### Function to send OpenAI requests using prompts

In [ ]:
# Define a function that prompts GPT to summarize and provide a risk summary for each command cluster
async def collect_risk_summaries(command_sample_size: int, system_prompt: str, user_prompt_template: str):
    # Rank the clusters by severity
    responses = []
    # Iterate each cluster of commands
    for cluster_number in clusted_commands_df["Cluster"].unique():
        if cluster_number == -1:
            # Skip unclustered data for now
            continue
        
        # Fetch the commands for just the current cluster
        _this_cluster = clusted_commands_df[clusted_commands_df["Cluster"] == cluster_number]
        # Grab a sample of the cluster (or all if less than sample size)
        _this_cluster_sample = _this_cluster if _this_cluster.shape[0] <= command_sample_size \
            else _this_cluster.sample(command_sample_size)

        # Create a string of all the sample commands to insert into the user prompt
        _cmd_list = ["\n".join([f"<command>", cmd, f"</command>"]) for cmd in _this_cluster_sample["Command"]]
        cmd_body = "\n\n".join(_cmd_list)

        # Craft the user prompt for this cluster of commands
        user_prompt = user_prompt_template.format(cluster_number=cluster_number, command=cmd_body)

        # Get the response and added it to the responses to be returned
        response = await client.chat.completions.create(
            model="gpt-4o-mini",  # <--- The OpenAI chat model to use
            messages=[
                {"role": "system", "content": system_prompt},  # <--- System prompt
                {"role": "user", "content": user_prompt}  # <--- User prompt
            ]
        )
        responses.append(response)
    # Return OpenAI Chat response
    return responses

# Create a cleanup function for parsing JSON

In [ ]:
# Create a function to parse json data from a response (sometimes a little cleaning is required)
def parse_json(opanai_response):
    # Fetch the content from the openai response
    content = opanai_response.choices[0].message.content
    # Search for a json response in the content
    json_str = re.search(r'(\{.*\})', content, re.DOTALL).group(1)
    # Strip single slashes in body (this is a common issue)
    json_str = re.sub(r'(?<!\\)\\(?!\\)', "\\\\\\\\", json_str)
    # Attempt to parse the json response
    data = None
    try:
        data = json.loads(json_str)
        try:
            data = dirtyjson.loads(json_str)
        except:
            pass
    except:
        raise Exception("Unable to parse a JSON response.")

    return data

## Collect Risk Responses into a DataFrame

In [ ]:
# Collect the risk responses from GPT
risk_summary_responses = await collect_risk_summaries(
    command_sample_size, system_prompt, user_prompt_template
)
# Create a DataFrame of the responses
rankings_df = pd.DataFrame([parse_json(r) for r in risk_summary_responses])
# We skipped unclustered data (-1) so start at index 1
rankings_df["cluster_number"] = clusted_commands_df["Cluster"].unique()[1:]

In [ ]:
rankings_df[rankings_df["cluster_number"]==79]

# Create an Excel Workbook!

In [ ]:
writer = pd.ExcelWriter('command_clusters.xlsx', engine='xlsxwriter')   
workbook=writer.book

_worksheet_r=workbook.add_worksheet('Cluster Rankings')
writer.sheets['Cluster Rankings'] = _worksheet_r
rankings_df.to_excel(
    writer, sheet_name='Cluster Rankings', 
    columns=["cluster_number", "risk_score", "cluster_description"], 
    startrow=0 , startcol=0, index=False
)

_worksheet_c=workbook.add_worksheet('Commands')
writer.sheets['Commands'] = _worksheet_c
clusted_commands_df.to_excel(
    writer, sheet_name='Commands', 
    columns=["Cluster", "Command"], 
    startrow=0 , startcol=0, index=False
)

writer.close()